# SageMaker/DeepAR Model Training Demo
This notebook:
1. Initializes the SageMaker environment
2. Loads the data from SageMaker Feature Store
3. Prepares the data for the model training
4. Trains an Amazon DeepAR time series forecasting model
5. Deploys a SageMaker Endpoint for testing the model
6. Cleans up the SageMaker model

__Credits:__
* This notebook is mainly inspired by the [SageMaker/DeepAR demo on electricity dataset notebook](https://github.com/aws/amazon-sagemaker-examples/blob/main/introduction_to_amazon_algorithms/deepar_electricity/DeepAR-Electricity.ipynb)

## Model Training
### Target Variable
By default, this notebook selects the `avg_fee_1min` (average transaction fee over a 1 minute period) aggregated variable generated by Kinesis Analytics as the target value for the model. `total_nb_trx_1min` (total number of transactions during a 1 minute period) and `total_fee_1min` (total amount of transaction fees during a 1 minute period) can also be used. 
### Time Series Period
In this demo, the data are aggregate in near real time by the Kinesis Analytics stream over a 1 minute period. When predicting, we train the model to predict 30 steps (30 minutes) into the future. Aggregating data over such a short time period and predicting only 30 minutes in the future might not be useful in a real world scenario. This is however done on purpose to generate enough data fast enough to train the model without having to let the demo run for weeks and months in order to collect enough data (especially due to the costs incured to running the demo).
### Why using Amazon DeepAR Model?
[Amazon DeepAR model](https://docs.aws.amazon.com/sagemaker/latest/dg/deepar.html) is a deep learning model use to forecast one-dimensional time series. It is best suited when *"you have many similar time series across a set of cross-sectional units"* e.g., many households electricity consumption, a server farm CPU utilization, temperatures from multiple weather stations.

In this demo we have a single time series of blockchain transactions and aggregate in near realtime some metrics over a 1 minute period by default. Our data is thus not optimized to be used with the Amazon DeepAR model.

So why do we chose Amazon DeepAR?

Despite not being an optimum model for the data we chose to use Amazon DeepAR model for the following reasons:
* The objective of this demo is not to train the best model but to demo MLOps on AWS
* According to AWS documentation, Amazon DeepAR can be used with single time series and based on basic comparison tests we did with forecasting models like ARIMA it provided better initial results
* Being a model developed by AWS, the SageMaker environment provides out of the box containers to train and deploy this model without having to build our own, thus simplifying the demo

In [ ]:
from __future__ import print_function

%matplotlib inline

import json
import os

import boto3
import sagemaker
import pandas as pd
import matplotlib.pyplot as plt
from sagemaker.session import Session
from sagemaker.feature_store.feature_group import FeatureGroup
from utils import DeepARPredictor, get_ssm_parameters

## Initialisation
### Retrieve Parameters
We collect here information from ssm parameters about:
* the SageMaker Feature Store storing our data
* model target
* model training hypertunning parameters

In [ ]:
def get_ssm_parameters(ssm_client, param_path):
    """Retrieves the SSM parameters from the specified path

    Args:
        ssm_client (botocore.client): The SSM client
        param_path (str): The path to the SSM parameters

    Returns:
        Dict[str, str]: The SSM parameters
    """
    parameters = {}
    try:
        response = ssm_client.get_parameters_by_path(
            Path=param_path, Recursive=False, WithDecryption=False
        )
        for param in response["Parameters"]:
            parameters[param["Name"].split("/")[-1]] = param["Value"]
        while next_token := response.get("NextToken"):
            response = ssm_client.get_parameters_by_path(
                Path=param_path,
                Recursive=False,
                WithDecryption=False,
                NextToken=next_token,
            )
            for param in response["Parameters"]:
                parameters[param["Name"].split("/")[-1]] = param["Value"]
    except Exception as e:
        print(f"An error occurred reading the SSM stack parameters: {e}")
    return parameters

In [ ]:
ssm_client = boto3.client("ssm")
# Read the Feature Group Name from SageMaker feature store
stack_parameters = get_ssm_parameters(ssm_client, "/rdi-mlops/stack-parameters")
# Read the SSM Parameters storing the parameters for the model training
target_parameters = get_ssm_parameters(ssm_client, "/rdi-mlops/sagemaker/model-build/target")
# Read the SSM Parameters storing the model training hyperparamters
model_training_hyperparameters = get_ssm_parameters(
    ssm_client, "/rdi-mlops/sagemaker/model-build/training-hyperparameters"
)

### SageMaker Initialisation

In [ ]:
# Set S3 Buckets variables
model_artifacts_bucket = f"{stack_parameters['project-prefix']}-sagemaker-experiment-{stack_parameters['bucket-suffix']}"
# Set prefix used for all data stored within the bucket
s3_model_prefix = "deepar-model"
train_validation_data_folder_prefix = "model-testing"
s3_data_path = (
    f"s3://{model_artifacts_bucket}/data/{train_validation_data_folder_prefix}"
)
s3_output_path = f"s3://{model_artifacts_bucket}/{s3_model_prefix}/output"

In [ ]:
# Set session variables
sagemaker_session = sagemaker.Session(default_bucket=model_artifacts_bucket)
region = sagemaker_session.boto_region_name
account_id = sagemaker_session.account_id()
role = sagemaker.get_execution_role()
boto_session = boto3.Session(region_name=region)

In [ ]:
# Set boto3 S3 client
s3 = boto3.resource("s3")
# Set feature store session
sagemaker_client = boto_session.client(service_name="sagemaker", region_name=region)
featurestore_runtime = boto_session.client(
    service_name="sagemaker-featurestore-runtime", region_name=region
)

feature_store_session = Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_client,
    sagemaker_featurestore_runtime_client=featurestore_runtime,
)

## Load Data From FeatureStore
The notebook uses Athena to load the data from the S3 bucket of the SageMaker Feature Store.

In [ ]:
transactions_feature_group = FeatureGroup(
    name=stack_parameters["sagemaker-feature-group-name"],
    sagemaker_session=feature_store_session,
)

In [ ]:
transactions_feature_group.describe()

Query the Data from FeatureStore using Athena

In [ ]:
transactions_data_query = transactions_feature_group.athena_query()
transactions_data_table = transactions_data_query.table_name

query_string = f'SELECT * FROM "{transactions_data_table}"'

# run Athena query. The output is loaded to a Pandas dataframe.
# dataset = pd.DataFrame()
transactions_data_query.run(
    query_string=query_string,
    output_location="s3://" + model_artifacts_bucket + "/query_results/",
)
transactions_data_query.wait()
df = transactions_data_query.as_dataframe()

Simple Dataset ETL:
- sort the values by time
- convert the time column from string to Pandas TImestamp

In [ ]:
df.sort_values(by="tx_minute", axis=0, ascending=True, inplace=True)
df["tx_minute"] = pd.to_datetime(df["tx_minute"])
df.set_index("tx_minute", drop=True, inplace=True)
df.head()

In [ ]:
print(f"Number of data points in the dataset: {len(df)}")

## Train, Test and Validation Splits
As mentioned earlier, by default the data are aggregated over a 1 minute period and the model is trained to predict 5 data points into the future.

According to the [documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/deepar.html):
>"Because lags are used, a model can look further back in the time series than the value specified for context_length. Therefore, you don't need to set this parameter to a large value. We recommend starting with the value that you used for prediction_length."

As a result we set `predition_length` (how far in the future we predict) to 5 and the `context_length` (how far in the past the model sees) to the same value.

### Amount of Data in each Dataset
By default we use 10 test windows for validation during the training, meaning that the total amount of data removed from the training dataset is `10*5=50` datapoints. We also keep 1 prediction window (`5` datapoints) to test the model at the end.

In such a case, the minimum amount of data is:
`how much data in the passt is used for prediction (context_length) + the prediction window size (predition_length) * (number of validation windows + 1 test window)`

In case there are not enough data, we set the prediction window size (`predition_length`) and validation window size (`test_length`) to 5% of the amount of data. How far in the past the model sees (`context_length`) is set to the remaining amount of data after keeping data for testing and validation.

We recommend to run the data ingestion long enough to at least have the minimum amount of data points.

### Train, Test and Validation Data
Based on the above and how the DeepAR model works the various datasets are splitted as follow:
* The training dataset is all datapoints minus the last `num_validation_windows * predition_length + test_length` datapoints, which by default will be `10*5+5=55` datappoints
* The validation datasets, like the training dataset, start at the beginning of the dataset. But each of the validation dataset is longer than the previous one by the `predition_length` window size
* The test data, is a set of data equivalent to the prediction window. For testing, the model will be given all data minus the `test_length` (which is equal to `predition_length`) to predict future values. The predicted values will be compared to the test data to check the model quality.

__Example:__
If the dataset has 100 datapoints with `predition_length = context_length = test_length = 5 datapoints` and 10 test windows, the datasets will be as follow:
* training data: [0:45]
* validation data #1: [0:50]
* validation data #2: [0:55]
* validation data #3: [0:60]
* ...
* validation data #10: [0:95]
* test data: [95:100]

In [ ]:
start_dataset = df.index.min()
end_dataset = df.index.max()
freq = "1min"
dataset_period = end_dataset - start_dataset
num_validation_windows = int(target_parameters["num_validation_windows"])
target_col = target_parameters["target_col"]

In [ ]:
total_nb_data_points = len(df)
prediction_length = int(target_parameters["prediction_length"])
context_length = prediction_length
test_length = prediction_length
min_data_length = context_length + prediction_length * (num_validation_windows + 1)

In [ ]:
if total_nb_data_points < min_data_length:
    prediction_length = int(total_nb_data_points * 0.05)
    test_length = prediction_length
    context_length = total_nb_data_points - prediction_length * (
        num_validation_windows + 1
    )
validation_windows_length = num_validation_windows * prediction_length

### Data Formatting
The data have to be formated to the DeepAR format as described in [this documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/deepar.html#deepar-inputoutput).

Data are of JSON format, following the this structure:
* the start time stamps (additional time stamps are derived from this start time stamps and the frequency parameter of the model)
* the target values, starting at the start time stamp

We do not use DeepAR's optional features and as such do not need other fields.

In [ ]:
df_test = df[-test_length:]
df_train_validation = df[:-test_length]
df_train = df_train_validation[:-validation_windows_length]

training_data = [
    {
        "start": str(start_dataset),
        "target": list(df_train[target_col]),
    }
]
validation_data = [
    {
        "start": str(start_dataset),
        "target": list(
            df_train_validation.iloc[
                0 : -int((num_validation_windows - k) * prediction_length),
                df_train_validation.columns.get_loc(target_col),
            ]
        ),
    }
    for k in range(1, num_validation_windows)
]
validation_data.append(
    {
        "start": str(start_dataset),
        "target": list(df_train_validation[target_col]),
    }
)

We write the the train and test datasets JSON dictionaries file to S3

In [ ]:
def write_dicts_to_file(path, data):
    with open(path, "wb") as fp:
        for d in data:
            fp.write(json.dumps(d).encode("utf-8"))
            fp.write("\n".encode("utf-8"))

In [ ]:
if not os.path.isdir(train_validation_data_folder_prefix):
    os.mkdir(train_validation_data_folder_prefix)
write_dicts_to_file(f"{train_validation_data_folder_prefix}/train.json", training_data)
write_dicts_to_file(
    f"{train_validation_data_folder_prefix}/validation.json", validation_data
)

In [ ]:
def copy_to_s3(local_file, s3_path, override=False):
    assert s3_path.startswith("s3://")
    split = s3_path.split("/")
    bucket = split[2]
    path = "/".join(split[3:])
    buk = s3.Bucket(bucket)

    if len(list(buk.objects.filter(Prefix=path))) > 0:
        if not override:
            print(f"File {s3_path} already exists.\nSet override to upload anyway.\n")
            return
        else:
            print("Overwriting existing file")
    with open(local_file, "rb") as data:
        print(f"Uploading file to {s3_path}")
        buk.put_object(Key=path, Body=data)

In [ ]:
copy_to_s3(
    f"{train_validation_data_folder_prefix}/train.json",
    s3_data_path + "/train/train.json",
    override=True,
)
copy_to_s3(
    f"{train_validation_data_folder_prefix}/validation.json",
    s3_data_path + "/validation/validation.json",
    override=True,
)

## Train a Model
### Initialize the Model

In [ ]:
image_name = sagemaker.image_uris.retrieve("forecasting-deepar", region)
estimator = sagemaker.estimator.Estimator(
    image_uri=image_name,
    sagemaker_session=sagemaker_session,
    role=role,
    instance_count=1,
    instance_type="ml.c5.2xlarge",
    base_job_name=f"{stack_parameters['project-prefix']}-deepar",
    use_spot_instances=True,
    max_run=60 * 60 - 1,
    max_wait=60 * 60,
    output_path=s3_output_path,
)

data_channels = {
    "train": f"{s3_data_path}/train/",
    "test": f"{s3_data_path}/validation/",
}

### Train a Model
The hyperparameters are set based on a hyperparameter tuning job we ran once. If you run yourself the hyperparameter tuning job you might get different values based on your data. But again the objective here is not to train the best model but to have a model to test MLOps on AWS SageMaker.

In [ ]:
hyperparameters = {
    "time_freq": freq,
    "epochs": model_training_hyperparameters["epochs"],
    "early_stopping_patience": model_training_hyperparameters[
        "early_stopping_patience"
    ],
    "mini_batch_size": model_training_hyperparameters["mini_batch_size"],
    "learning_rate": model_training_hyperparameters["learning_rate"],
    "likelihood": model_training_hyperparameters["likelihood"],
    "dropout_rate": model_training_hyperparameters["dropout_rate"],
    "embedding_dimension": model_training_hyperparameters["embedding_dimension"],
    "context_length": str(context_length),
    "prediction_length": str(context_length),
}
estimator.set_hyperparameters(**hyperparameters)

In [ ]:
estimator.fit(inputs=data_channels, wait=True)

## Create Endpoint and Predictor
A utility class is created to query the endpoint and perform predictions. This class is mainly copied from the AWS [DeepAR demo notebook](https://github.com/aws/amazon-sagemaker-examples/blob/main/introduction_to_amazon_algorithms/deepar_electricity/DeepAR-Electricity.ipynb)

__Note: Remember to delete the endpoint after running this experiment. A cell at the very bottom of this notebook will do that: make sure you run it at the end.__

In [ ]:
predictor = estimator.deploy(
    initial_instance_count=1, instance_type="ml.m5.large", predictor_cls=DeepARPredictor
)

## Make Predictions and Plot Results
We convert the train-test data to a Pandas Series and we use it to predict data points as defined in `prediction_length`.
The predicted data are compared to the validation dataset to evaluate the model.

In [ ]:
# Convert the context data to time series
ts = df.iloc[:-test_length, df.columns.get_loc(target_col)]
# Convert the validation dataframe to a time series
target_ts = df.iloc[-test_length:, df.columns.get_loc(target_col)]

df_predictions = predictor.predict(ts=ts, quantiles=[0.10, 0.5, 0.90])

In [ ]:
def plot(
    predictor,
    ts,
    target_ts,
    freq=pd.Timedelta(1, "min"),
    cat=None,
    show_samples=False,
    confidence=80,
):
    forecast_start_date = target_ts.index[0]
    print(
        "calling served model to generate predictions starting from {}".format(
            str(forecast_start_date)
        )
    )
    assert confidence > 50 and confidence < 100
    low_quantile = 0.5 - confidence * 0.005
    up_quantile = confidence * 0.005 + 0.5

    # we first construct the argument to call our model
    args = {
        "ts": ts,
        "return_samples": show_samples,
        "quantiles": [low_quantile, 0.5, up_quantile],
        "num_samples": 100,
    }

    plt.figure(figsize=(20, 3))
    ax = plt.subplot(1, 1, 1)

    if cat is not None:
        args["cat"] = cat
        ax.text(0.9, 0.9, "cat = {}".format(cat), transform=ax.transAxes)

    # call the end point to get the prediction
    prediction = predictor.predict(**args)

    # plot the samples
    if show_samples:
        for key in prediction.keys():
            if "sample" in key:
                prediction[key].plot(
                    color="lightskyblue", alpha=0.2, label="_nolegend_"
                )

    # plot the target
    target_ts.plot(color="black", label="target")

    # plot the confidence interval and the median predicted
    ax.fill_between(
        prediction[str(low_quantile)].index,
        prediction[str(low_quantile)].values,
        prediction[str(up_quantile)].values,
        color="b",
        alpha=0.3,
        label="{}% confidence interval".format(confidence),
    )
    prediction["0.5"].plot(color="b", label="P50")
    ax.legend(loc=2)

    # fix the scale as the samples may change it
    ax.set_ylim(target_ts.min() * 0.5, target_ts.max() * 1.5)

In [ ]:
plot(
    predictor,
    ts=ts,
    target_ts=target_ts,
    show_samples=False,
    confidence=60,
)

## Cleanup
Delete the model & endpoint used for testing.

In [ ]:
predictor.delete_model()
predictor.delete_endpoint()